# capturing esm code in playwright python

Sometimes I really need to run JavaScript code and I don't really care to use node. I'm looking for a solution where I can run JavaScript code using imports. This means I need to be able to execute ESM code and control. What is returned from it.

The system that I'm building with already has `playwright` so I want to figure out a way to use that as my application interface. The best case scenario for me is that I can run ESM code with imports and go directly to data frame objects.

In [143]:
%%
## building a search index with `lunr`

An example of this application is building a JavaScript search index.
To start we need to load in data that has text. 

    df = Index(Path().glob("*.ipynb")).to_series().apply(
        compose_left(Path.read_text, json.loads)
    ).apply(Series).cells.apply(Series).stack().apply(Series)

Then we need to build the index that we need to invert for our search results.

    index = DataFrame(
        df.index.to_series().apply(compose_left(partial(map, str), "#".join)).pipe(Index, name="id")
    ).assign(source=df.source.apply("".join).values).pipe(lambda x: x[x.source.astype(bool)])

{{index.T._repr_html_()}}

,0,4,5,6,7,8,9,10,11,12,...,1185,1186,1187,1188,1189,1193,1194,1195,1196,1197
id,Untitled10.ipynb#0,Untitled34.ipynb#0,Untitled34.ipynb#1,Untitled34.ipynb#2,Untitled34.ipynb#3,Untitled34.ipynb#4,Untitled34.ipynb#5,Untitled34.ipynb#6,Untitled34.ipynb#7,Untitled34.ipynb#8,...,Untitled40.ipynb#12,Untitled40.ipynb#13,Untitled40.ipynb#14,Untitled40.ipynb#15,Untitled40.ipynb#16,Untitled38.ipynb#0,Untitled21.ipynb#0,Untitled21.ipynb#1,Untitled21.ipynb#2,Untitled21.ipynb#3
source,import,from tonyfast.tonyfast.xxiv.schema_frame impor...,@pandas.core.accessor.register_dataframe_acces...,%env PRINT 0,"HTML(df.apply(dict, axis=1).repr.to_items(""li""...","HTML(df.repr.to_items(""dd"").object.stack().rep...",HTML(df.groupby([0]*len(df)).apply(\n lambd...,rep = df.repr\nrep.schema = rep.schema.combine...,rep = df.repr\nrep.schema = rep.schema.combine...,rep = df.repr\nrep.schema = rep.schema.combine...,...,%%\n\n style=\\n```css\ntextarea[name=sourc...,"tocs = cell_lists.apply(lambda x: x.select(""h1...","mains = tocs.to_frame().join(cell_lists, how=...","pages = Series(None, mains.index).fillna("""").h...","pages.html.select_one(""main"").html.interact()",%%\n<form>\n <button>edit style</button>\n ...,import graphviz,%%\n\n tpl =\\n```dot\ngraph LR {\n layo...,display(*(graphviz.Source(\n tpl%x\n) for x...,"%% --md\n```mermaid\n%% %%{init: {""flowchart"":..."


In [144]:
%%
## running esm code with through python playwright

    async def run_esm(code, playwright=None, browser=None, page=None):
run javascript esm `code` using a `playwright` instance, `browser`, and `page`
        
        async with __import__("contextlib").AsyncExitStack() as stack:
the example we're showing is a one shot execution, but there are situations where 
a `playwright`, `browser`, or `page` might reused for executing multiple blocks of code.
            
            if playwright is None: 
                playwright = await stack.enter_async_context(__import__("playwright.async_api").async_api.async_playwright())
            if browser is None: browser = await playwright.firefox.launch()
            if page is None: page = await browser.new_page()
            async with page.expect_console_message() as msg_info:
this is the magic trick to running our code, we must put something in the console. we wait for message from the console
and catch it in python space
                
                await page.add_script_tag(type="module", content=code)
            return await asyncio.gather(*(x.json_value() for x in (await msg_info.value).args))

running esm code with through python playwright 
 async def run_esm(code, playwright=None, browser=None, page=None):
 
 run javascript esm code using a playwright instance, browser , and page 
 async with __import__("contextlib").AsyncExitStack() as stack:
 
 the example we're showing is a one shot execution, but there are situations where
a playwright , browser , or page might reused for executing multiple blocks of code. 
 if playwright is None: 
 playwright = await stack.enter_async_context(__import__("playwright.async_api").async_api.async_playwright())
 if browser is None: browser = await playwright.firefox.launch()
 if page is None: page = await browser.new_page()
 async with page.expect_console_message() as msg_info:
 
 this is the magic trick to running our code, we must put something in the console. we wait for message from the console
and catch it in python space 
 await page.add_script_tag(type="module", content=code)
 return await asyncio.gather(*(x.json_value() for x in (await msg_info.value).args))

In [145]:
%%
## building an actual search index

`lunr_search_index` is the esm code we want to run.

    lunr_search_index = (
```js
import lunr from "https://esm.sh/lunr"

var documents = %s;
var idx = lunr(function () {
    this.ref('id'); this.field('source')
    documents.forEach(function (doc) {this.add(doc)}, this)})

// this console message is caught by the playwright context manager
console.log(idx)
```                                                            
    % index.to_json(orient="records"))

create the `inverted_index` with `run_esm`, our demo function.

    inverted_index = await run_esm(lunr_search_index)

{{Series(inverted_index).apply(Series)._repr_html_()}}

,invertedIndex,fieldVectors,tokenSet,fields,pipeline
0,"{'0': {'_index': 107, 'source': {'Untitled34.i...",{'source/Untitled10.ipynb#0': {'_magnitude': 0...,"{'final': True, 'edges': {'0': {'final': True,...",[source],{'_stack': [None]}
